In [2]:
import re
from pathlib import Path
from datetime import datetime
import time
import os
from tqdm import tqdm
from itertools import groupby
from operator import itemgetter

In [16]:
def prepare_whatsapp_data(directory, spell_dict = None):
    def get_date_from_msg(msg):
        date = re.findall(r"(\d+/\d+/\d+, \d+:\d+) -", msg)

        if date:
            return time.mktime(datetime.strptime(date[0], r"%m/%d/%y, %H:%M").timetuple())
        else:
            return 0
        
    def get_sorted_sequences(filename):
        f = open(filename, "r", encoding = "utf-8")
        messages = []
        index = 0
        for line in f:
            if re.match(r"\d+/\d+/\d+, \d+:\d+", line):
                messages.append(line)
                index += 1
            else:
                messages[index - 1] += line

        messages.sort(key = get_date_from_msg)

        return messages
    
    def separate_emojis(text):
        # this matches emojis which are directly preceded by non-whitespace chars
        emoji_pattern = re.compile(r"(\S*?)" u"([\U00010000-\U0010ffff(\u2764\uFE0F)])")
        # insert blank in front of emoji
        text = re.sub(emoji_pattern, r"\1 \2", text)
        # this matches emojis which are directly succeeded by non-whitespace chars
        emoji_pattern = re.compile(u"([\U00010000-\U0010ffff(\u2764\uFE0F)])" r"(\S+)")
        # insert blank behind emoji
        text = re.sub(emoji_pattern, r"\1 \2", text)
        
        return text
    
    def replace_multichars(text):
        new = text
        multichars = re.findall(r'((\w)\2{2,})', text)
        if multichars:
            for char in multichars:
                new = new.replace(char[0], char[1] * 2)

        return new     

    def replace_digits(text):
        text = re.sub(r"\d+", "<number>", text)
        
        return text
    
    for filename in Path(directory).glob("*.txt"):
        # get head and tail of path
        split = os.path.split(filename)
        # create new "edited" directory
        path = os.path.join(split[0], "edited")
        # create directory if not existant
        if not os.path.exists(path):
            os.mkdir(path)
        # start a new file for the current whatsapp chat
        with open(os.path.join(split[0], "edited", split[1]), "w+", encoding = "utf-8") as f:
            # get list of messages sorted by time
            messages = get_sorted_sequences(filename)
            # get list of tuples of author and message for every message
            messages = [(author, text.lower().strip()) for msg in messages for author, text in re.findall(r".+? - (.+?): (.+)", msg, flags = re.DOTALL)]
            # group consecutive messages of the same author
            groups = groupby(messages, key = itemgetter(0))
            # get list of grouped messages where consecutive texts of one author are separated by "<new>""
            messages = [[msg.strip() for author, msg in group] for i, group in groups]
            # separate emojis with spaces
            messages = [[separate_emojis(msg) for msg in group] for group in messages]
            # replace multiple consecutive characters with 2 of those chars
            messages = [[replace_multichars(msg) for msg in group] for group in messages]
            # replace digits with a special <number> token
            messages = [[replace_digits(msg) for msg in group] for group in messages]
            # always write two consecutive messages separated by a tab into the file
            for i in range(len(messages) - 1):
                f.write(str(messages[i]) + "\t" + str(messages[i + 1]) + "\n")
            

In [20]:
def prepare_movie_dialogues(path_to_lines, path_to_dialogues):
    movie_lines = {}
    with open(Path(path_to_lines), "r", encoding = 'iso-8859-1') as f:
        for line in f:
            line_id, _, _, _, text = line.split("+++$+++")
            line_id = line_id.strip()
            text = text.lower().strip()
            movie_lines[line_id] = text
    
    dialogues = []
    
    with open(Path(path_to_dialogues), "r", encoding = 'iso-8859-1') as f:
        for line in f:
            _, _, _, dialogue = line.split("+++$+++")
            dialogue = eval(dialogue)
            dialogues.append(dialogue)
            
    base_path = os.path.split(path_to_lines)[0]
    new_file_path = os.path.join(base_path, "movie-dialogues-edited.txt")
    
    with open(new_file_path, "w+", encoding = "utf-8") as f:
        for dialogue in dialogues:
            for i in range(len(dialogue) - 1):
                line1 = movie_lines[dialogue[i]]
                line2 = movie_lines[dialogue[i + 1]]
                f.write(str([line1]) + "\t" + str([line2]) + "\n")

In [21]:
prepare_movie_dialogues("movie-corpus/movie_lines.txt", "movie-corpus/movie_conversations.txt")

In [17]:
prepare_whatsapp_data("chats")